In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup, CData
import requests
import html
import json
from time import sleep
import time
from pandas.io.json import json_normalize
from urllib.request import urlopen
import re
import pandas as pd 
import numpy as np

In [2]:
driver = webdriver.Chrome("/Applications/chromedriver")

In [3]:
hashtags = ['végétarien', 'recettes', 'bienmanger', 'reequilibrage', 'dietetique', 'recettemaison', 'minciravecplaisir',
           'repashealthy', 'recetteminceur', 'mangerequilibre', 'instaregime', 'recettefacile', 'recetterapide', 'saladecomposée',
           'mangersainement', 'mangerbien', 'instaregimeuse', 'reequilibragealimentaire', 'mangersain', 'recettedujour',
           'mangermieux', 'ideerecette', 'cuisinesaine', 'aimetarecette']



In [4]:
links=[]
for hashtag in hashtags:
    driver.get('https://www.instagram.com/explore/tags/'+hashtag)
    time.sleep(5) 
    s0 = 0
    s1 = 1.5
    for i in range(4):       
        Pagelength = driver.execute_script("window.scrollTo({}, document.body.scrollHeight/{});".format(s0, s1))
        source = driver.page_source
        data=BeautifulSoup(source, 'html.parser')
        body = data.find('body')
        script = body.find('span')
        for link in script.findAll('a'):
             if re.match("/p", link.get('href')):
                links.append('https://www.instagram.com'+link.get('href'))
        #sleep time is required. If you don't use this Instagram may interrupt the script and doesn't scroll through pages
        time.sleep(5)
        s0 = s1
        s1 = (s1 + 1.5)

print(len(links))
print(len(set(links)))
links = set(links)
links = list(links)    
    
        

3942
797


In [5]:
scraper_result = pd.DataFrame()
for i in range(len(links)):
    try:
        page = urlopen(links[i]).read()
        data = BeautifulSoup(page, 'html.parser')
        body = data.find('body')
        script = body.find('script')
        raw = script.text.strip().replace('window._sharedData =', '').replace(';', '')
        json_data = json.loads(raw)
        if json_data['entry_data']['PostPage'][0]['graphql']['shortcode_media']['is_video'] == False:
            post = json_data['entry_data']['PostPage'][0]['graphql']['shortcode_media']['edge_media_to_caption']['edges'][0]['node']
            post = json.dumps(post)
            post = json.loads(post)
            post = json_normalize(post) 
            post['caption'] = json_data['entry_data']['PostPage'][0]['graphql']['shortcode_media']['accessibility_caption']
            post['timestamp'] = json_data['entry_data']['PostPage'][0]['graphql']['shortcode_media']['taken_at_timestamp']
            post['username'] = json_data['entry_data']['PostPage'][0]['graphql']['shortcode_media']['owner']['username']
            post['likes_count'] = json_data['entry_data']['PostPage'][0]['graphql']['shortcode_media']['edge_media_preview_like']['count'] 
            post['link'] = links[i]
            post['image_link'] = json_data['entry_data']['PostPage'][0]['graphql']['shortcode_media']['display_url']
            x = pd.DataFrame.from_dict(post, orient='columns') 
            x.columns =  x.columns.str.replace("shortcode_media.", "")
            scraper_result = scraper_result.append(x)
       
    except:
        np.nan
#Just check for the duplicates
scraper_result.index = range(len(scraper_result.index))

print(len(scraper_result))
print(len(scraper_result.loc[scraper_result['caption'].str.contains("food")]))


621
457


In [79]:
scraper_result = result

In [6]:
# Create a dataframe with posts from the scraper
food_posts = scraper_result[scraper_result['caption'].str.contains("food")].copy()
food_posts['hashtags'] = food_posts.text.apply(lambda x: re.findall(r'[#@][^\s#@]+', x))
print(len(food_posts))

457


In [7]:
# Clean posts to prepare for text classification
import re
import unidecode

clean_texts = []
for index, row in food_posts.iterrows():
    clean_post = row.text
    # Remove hashtags
    hashtags = [hashtag[1:] for hashtag in row.hashtags]
    clean_post = ', '.join(str(word) for word in [w for w in clean_post.split('#') if w.strip() not in hashtags])   
    # Convert to unicode and remove accents
    clean_post = unidecode.unidecode(clean_post)
    # Remove Emails
    clean_post = re.sub('\S*@\S*\s?', "", str(clean_post))
    # Remove new line characters
    #clean_post = re.sub('\s+', " ", str(clean_post))
    # Remove distracting single quotes
    clean_post = re.sub("\'", "", str(clean_post))
    # Remove return line characters
    #clean_post = re.sub('\n', " ", str(clean_post))
    # Remove non alphabetical characters
    #clean_post = re.sub(r'[^A-Za-z]', " ", str(clean_post))
    # Lower all characters
    clean_post = str(clean_post).lower()
    
    clean_texts.append(clean_post)
    

food_posts = food_posts.assign(clean_text=clean_texts)
food_posts = food_posts[food_posts.clean_text != '']



In [8]:
# Save posts in a csv file

with open('instagram_food_posts.csv', 'w') as f:        
    food_posts.to_csv(f, header=True)

In [184]:
# Get list of posts with a link to the recipe in user's bio
patterns =['ma bio', 'notre bio', 'en bio', 'la bio']
indexes = []

for index, row in food_posts.iterrows():
    for pattern in patterns:
        if pattern in row.text:
            indexes.append(index)

link_posts = food_posts.loc[indexes]

